# MP2- Traffic Sign Classifier
Before you start, please read through this tutorial of Pytorch: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py.
The basic structure of this MP will be very similar to the example code in the tutorial. But your neural network model need to be more complicated. 

If any packege is not installed, you shall be able to install them through pip3 install. 

In [1]:
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io, transform
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, sampler
import torchvision
from torchvision import transforms, utils
import torchvision.utils as vutils
import torchvision.datasets as dset
from torch.autograd import Variable

Set up the parameters. If you have GPU, change the num_gpu. 

In [2]:
# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. The image size of original images vary from 15 to 250. 
#But to train our CNN we must have the fixed size for the inputs.
#All images will be resized to this size using a transformer.
image_size = 32

# Number of training epochs
num_epochs = 15

#number of training classes
num_classes = 43

# Learning rate for optimizers
learning_rate = 0.002

# Number of GPUs available. Use 0 for CPU mode.
num_gpu = 0

# Set random seed for reproducibility
manualSeed = 999
torch.manual_seed(manualSeed)

### Data Preprocessing 
To improve the performance of our CNN, we need to use some data augmentation techniques. We can change the orientation, location, scale, saturation or brightness of the original image to modify the original images or create new training images. 

The easiest way to go is openCV. If you are having problem with ppm format, you can convert the images to other image format (PNG, JPG...) using pillow packege: https://pillow.readthedocs.io/en/latest/handbook/tutorial.html

You are reqiured to test at least 3 techniques and compare the effectiveness(the accuracy improvement). For each technique, you need to generate new images for at least one class of images. For instance, if you originally have 300 training images in class 01, then you need to generate 300 new images and save them into the same folder as old images. Note that it may be helpful to name the new images properly, so if you find your technique is not helpful, you can remove them easier. <br>
You can skip this part and try to have a working neural network first, then come back here to finish it. 

### Load the data
Load the dataset into PyTorch. Assign the path to the training and testing dataset you downloaded previously to the variables "trainingClassifierRoot" and "testClassifierRoot" below. In the beginning you can just use the original data downloaded from website. Later on you will need to process the image before load them.  

In [3]:
##TODO
trainingClassifierRoot = '..\mp2_data\GTSRB_Final_Training_Images\GTSRB\Final_Training\Images'
testClassifierRoot = '../mp2_data/GTSRB_Online-Test-Images/GTSRB/Online-Test'
####

# Create the dataset
trainClassifierDataset = dset.ImageFolder(root=trainingClassifierRoot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

testClassifierDataset = dset.ImageFolder(root=testClassifierRoot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

# Create the dataloader
trainClassifierLoader = torch.utils.data.DataLoader(trainClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)
testClassifierLoader = torch.utils.data.DataLoader(testClassifierDataset, batch_size=batch_size, shuffle=True, num_workers=workers)

# Decide which device(GPU or CPU) we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

print ('...')

...


### Design the model
In this part you will design the structure of CNN. The basic syntax is the same as the toy example in PyTorch tutorial. But you definitely need some more complex models to meet the reqiurements. You need to try at least 3 different structures and report their accuracy and running time (how long it would take to go through the test set). The best one is required to achieve 97% accuracy. Describe the CNN structures and compare the results in your report. Feel free to use any existing structure you find on Internet, but citation in your report is needed.<br>
Here is a NN structure that has 98% accuracy. You can start from there: https://chatbotslife.com/german-sign-classification-using-deep-learning-neural-networks-98-8-solution-d05656bf51ad<br>
If you really want to get higher accuracy, here are some state-of-the-art papers for your reference: 
<li>https://arxiv.org/abs/1511.02992</li>
<li>https://www.sciencedirect.com/science/article/pii/S0893608018300054?via%3Dihub</li>
<li>http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf</li>

In [42]:
class TrafficSignClassifier(nn.Module):
    def __init__(self, ngpu):
        super(TrafficSignClassifier, self).__init__()
        self.ngpu = num_gpu
       
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, padding=(2,2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=(2,2))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=(2,2))

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=(2,2))
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding=(2,2))
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, padding=(2,2))

        self.pool = nn.MaxPool2d(2, 2)

        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(256)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.relu = nn.ReLU(inplace = True)
        self.linear = nn.Linear(4*4*64, num_classes)
        ####
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x  = self.pool(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)
        x = self.linear(x) 
        return x
print ('...')

...


In [43]:
classifier = TrafficSignClassifier(num_gpu).to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (num_gpu > 1):
    classifier = nn.DataParallel(classifier, list(range(num_gpu)))
    
#To load the trained weights
#Uncomment the following line to load the weights. Then you can run testing directly or continue the training
#classfier.load_state_dict(torch.load('./MP2weights.pth', map_location='cpu')) 
#classfier.eval()

#Print the model
print(classifier)

TrafficSignClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv6): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track

### Set up training environment
You can choose different loss functions and optimizers. Here I just use the same ones as in PyTorch official tutorial.

In [44]:
#training parameters
##TODO
criterion = nn.CrossEntropyLoss()#loss function
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
####

Now let's start the training. Just like in tutorial, you can print out your loss and the running time at the end of each epoch to monitor the training process. 

In [ ]:
# Training Loop, no need to run if you already loaded the weights
for epoch in range(1):  # loop over the dataset multiple times  
    running_loss = 0.0
    for i, data in enumerate(trainClassifierLoader, 0):
        # get the inputs
        inputs, labels = data

        # zero out the parameter gradients
        #Every time a variable is back propogated through, the gradient will be accumulated instead of being replaced. 
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)

        #loss.backward() computes dloss/dx for every parameter x
        loss.backward()

        #optimizer.step updates the value of x using the gradient x.grad.
        optimizer.step() 

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    
print('Finished Training')

### Test the Model
Now our model training is finished. If you are satisfied by the result from part of the test set, let's try it on all testing images. Print out the accuracy on all test images. Note you need to achieve 97% accuracy to get full grade.

In [3]:
## TODO

####

### Analyze the Results for Each Individual Class
To help futher improve the accuracy, we want to know for which classes our NN works well and for which classes it fails. Print out the accuracy of your classifier on each class on the whole testing dataset. Try to explain why some classess have low accuracy in your report. You don't have to speficy the name of each class. Just use something like "class 0" is good enough. 

In [4]:
## TODO


####

### Save Trained Weights
Notice that the trained weights are just variables right now and will be lost when you close the Jupyter file. Obviously, you don't want to train the model again and again. PyTorch can help you save your work. Read the "Saving & Loading Model for Inference" part from this tutorial: https://pytorch.org/tutorials/beginner/saving_loading_models.html.<br>
Please save the weights from your best model into "MP2weights.pth" and include it in your submission. TAs will not train the CNN for you. So if we cannot find this file, you will lose a lot of points.  

In [10]:
#save the weights into "./MP2weights.pth"
torch.save(classfier.state_dict(), "./MP2weights.pth")

### Tips for Advanced Techniques
Congratulations! Now we have built a wonderful trffic sign classifier. If you are unsatified about the accuraccy. Here are a few tricks that can help you improve the results:<br>
<li>Balance the dataset. Currently, the size of image dataset from different types of traffic signs vary from 100 to 2000. Hence, our CNN will be trained to be very good at classify those types with a lot of image samples, but behave poorly on the rest of types. One possible solution is to generate more image data from the "unfavorable" types to make each type to have similar number of image samples. </li>
<li>The selection of loss functions and optimizaters  and the learning rate could make a difference. Here is a good reference for different types of loss functions: https://isaacchanghau.github.io/post/loss_functions/</li>
<li>Training the model for too long will make our CNN overfit the training set. If that is the case, we could change the number of epoches to avoid it. </li>
<li>There are piles of articles online that teaches you how to improve your CNN. Do some Google search yourself and try implement some interesting ones.</li>

### Break It If You Can!
Now if you already have trained a neural network with accuracy higher than 90%, let's try to break our system by manipulating the test set. <br>
Find the smallest amount of salt and pepper (recall from lecture 2) noise that has to be added to any image in the data set that was classified correctly, for the image with noise to be miss-classified. Write a function to find this smallest noise for mis-classification, for each image class. Then explore how the smallest noise changes across the different classes. Perform the same experiment with gaussian noise instead of salt and pepper noise. What conclusions can you draw about robustness of your classifier from these experiments? <br>
Note: In this part you only need to pick one single image and test your code on it. Please put the original image and the image after adding noises into your report. 

In [ ]:
##TODO

####